<a href="https://colab.research.google.com/github/AtomGen-2/recommendor/blob/main/RecommendorV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install turicreate

     |████████████████████████████████| 92.0 MB 5.4 kB/s 
     |████████████████████████████████| 3.6 MB 47.3 MB/s 
     |████████████████████████████████| 86.4 MB 60 kB/s 
     |████████████████████████████████| 3.5 MB 16.7 MB/s 
     |████████████████████████████████| 322 kB 58.8 MB/s 
     |████████████████████████████████| 18.3 MB 75 kB/s 
     |████████████████████████████████| 449 kB 45.3 MB/s 
     |████████████████████████████████| 50 kB 4.1 MB/s 
     |████████████████████████████████| 2.9 MB 45.3 MB/s 
     |████████████████████████████████| 20.1 MB 2.1 MB/s 
     |████████████████████████████████| 3.8 MB 39.2 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=b03c2788d29aba3738b48a3337289a1725199f2210fa966722d78e12d46f5abf
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320860 sha256=b8937dc7dd3

In [5]:
pip install scripts

In [6]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
# import scripts.data_layer as data_layer
import scripts

In [10]:
import requests
url = 'https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/recommend_1.csv'
res = requests.get(url, allow_redirects=True)
with open('recommend_1.csv','wb') as file:
    file.write(res.content)
customers = pd.read_csv('recommend_1.csv')

In [11]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [14]:
import requests
url = 'https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/trx_data.csv'
res = requests.get(url, allow_redirects=True)
with open('trx_data.csv','wb') as file:
    file.write(res.content)
transactions = pd.read_csv('trx_data.csv')

In [15]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [16]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [17]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [18]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

In [19]:
print(data.shape)
data.head

(133585, 3)


<bound method NDFrame.head of         customerId  productId  purchase_count
0                0          1               2
1                0         13               1
2                0         19               3
3                0         20               1
4                0         31               2
...            ...        ...             ...
133580       28596        211               3
133581       28596        255               1
133582       28598        212               1
133583       28604        282               1
133584       28605         92               1

[133585 rows x 3 columns]>

In [20]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [21]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [22]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [23]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()


(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [24]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [25]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [26]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [30]:

# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [31]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23322 users and 300 items.

Data prepared in: 0.169642s

106868 observations to process; with 300 unique items.

In [32]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)


recommendations finished on 1000/62483 queries. users per second: 106929

recommendations finished on 2000/62483 queries. users per second: 108998

recommendations finished on 3000/62483 queries. users per second: 113114

recommendations finished on 4000/62483 queries. users per second: 113911

recommendations finished on 5000/62483 queries. users per second: 106505

...

recommendations finished on 59000/62483 queries. users per second: 103917

recommendations finished on 60000/62483 queries. users per second: 103771

recommendations finished on 61000/62483 queries. users per second: 103202

recommendations finished on 62000/62483 queries. users per second: 102962

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    132    | 3.109090909090909  |  1   |
|     0      |     37    | 3.015748031496063  |  2   |
|     0      |     34    |       3.004        |  3   |
|     0      |     0     | 2.948912015137181  |  4   |
|     0      |     27    | 2.809917355371901  |  5   |
|     0      |    110    | 2.8012048192771086 |  6   |
|     0      |     3     | 2.794979079497908  |  7   |
|     0      |    248    | 2.7209302325581395 |  8   |
|     0      |     32    | 2.6515151515151514 |  9   |
|     0      |    230    | 2.6390977443609023 |  10  |
|     1      |    132    | 3.109090909090909  |  1   |
|     1      |     37    | 3.015748031496063  |  2   |
|     1      |     34    |       3.004        |  3   |
|     1      |     0     | 2.948912015137181  |  4   |
|     1      |     27    | 2.809917355371901  |  5   |
|     1   

In [33]:

# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [34]:

# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [35]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23312 users and 300 items.

Data prepared in: 0.158491s

...

recommendations finished on 59000/62483 queries. users per second: 116368

recommendations finished on 60000/62483 queries. users per second: 116068

recommendations finished on 61000/62483 queries. users per second: 116017

recommendations finished on 62000/62483 queries. users per second: 114979

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     8     |  1.0  |  1   |
|     0      |    209    |  1.0  |  2   |
|     0      |    179    |  1.0  |  3   |
|     0      |    263    |  1.0  |  4   |
|     0      |     61    |  1.0  |  5   |
|     0      |    135    |  1.0  |  6   |
|     0      |     46    |  1.0  |  7   |
|     0      |     79    |  1.0  |  8   |
|     0      |    243    |  1.0  |  9   |
|     0      |    217    |  1.0  |  10  |
|     1      |    123    |  1.0  |  1   |
|     1      |     8     |  1.0  |  2   |
|     1      |    209    |  1.0  |  3   |
|     1      |    179    |  1.0  |  4   |
|     1      |    263    |  1.0  |  5   |
|     1      |     1     |  1.0  |  6   |
|     1      |     46    |  1.0  |  7   |
|     1      |     79    |  1.0  |  8   |
|     1      |    243    |  1.0  |  9   |
|     1      |    217    |  1.0  |  10  |
|     2      |    209    |  1.0  |

In [36]:

name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23265 users and 300 items.

Data prepared in: 0.152648s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/62483 queries. users per second: 96089.2

...

recommendations finished on 59000/62483 queries. users per second: 109267

recommendations finished on 60000/62483 queries. users per second: 109220

recommendations finished on 61000/62483 queries. users per second: 108880

recommendations finished on 62000/62483 queries. users per second: 109175

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    226    |  0.7793103448275862 |  1   |
|     0      |    247    | 0.32782369146005513 |  2   |
|     0      |    230    |  0.3248120300751874 |  3   |
|     0      |    125    | 0.24857142857142825 |  4   |
|     0      |    248    | 0.24431818181818182 |  5   |
|     0      |    294    | 0.24328358208955192 |  6   |
|     0      |    204    | 0.22417582417582405 |  7   |
|     0      |    213    | 0.22222222222222232 |  8   |
|     0      |     72    | 0.21868787276341947 |  9   |
|     0      |     83    |  0.217948717948718  |  10  |
|     1      |    226    |  0.7793103448275862 |  1   |
|     1      |    247    | 0.32782369146005513 |  2   |
|     1      |    230    |  0.3248120300751874 |  3   |
|     1      |    125    | 0.24857142857142825 |  4   |
|     1      |    248    | 0.24431818181818182 |

In [49]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23322 users and 300 items.

Data prepared in: 0.161857s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 70553.8

recommendations finished on 60000/62483 queries. users per second: 70462.5

recommendations finished on 61000/62483 queries. users per second: 70639

recommendations finished on 62000/62483 queries. users per second: 70511.4

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |     2     | 0.13316982984542847 |  1   |
|     0      |     5     |  0.1055589497089386 |  2   |
|     0      |     21    | 0.08901822566986084 |  3   |
|     0      |     33    | 0.08073817491531372 |  4   |
|     0      |    157    | 0.07760765552520751 |  5   |
|     0      |     13    | 0.07526716589927673 |  6   |
|     0      |     17    |  0.0706416666507721 |  7   |
|     0      |     19    | 0.06861926317214966 |  8   |
|     0      |     59    | 0.06720507144927979 |  9   |
|     0      |     0     |  0.0667323648929596 |  10  |
|     1      |     14    | 0.16740621626377106 |  1   |
|     1      |     1     | 0.16053596511483192 |  2   |
|     1      |     17    | 0.12358514592051506 |  3   |
|     1      |     5     | 0.11797770112752914 |  4   |
|     1      |     39    |  0.1091022863984108 |

In [50]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23312 users and 300 items.

Data prepared in: 0.145391s

...

recommendations finished on 59000/62483 queries. users per second: 69916.6

recommendations finished on 60000/62483 queries. users per second: 69780.6

recommendations finished on 61000/62483 queries. users per second: 69375.5

recommendations finished on 62000/62483 queries. users per second: 69246.7

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     0      |     2     | 0.08897279067472978  |  1   |
|     0      |     5     | 0.06620296023108742  |  2   |
|     0      |     15    | 0.055364234880967575 |  3   |
|     0      |     8     | 0.055201839316974984 |  4   |
|     0      |     0     | 0.050290487029335716 |  5   |
|     0      |     9     | 0.04979045824571089  |  6   |
|     0      |     61    | 0.048255980014801025 |  7   |
|     0      |     14    | 0.04655404524369673  |  8   |
|     0      |     12    | 0.045837023041465065 |  9   |
|     0      |     25    | 0.04491797902367332  |  10  |
|     1      |     11    | 0.07659927010536194  |  1   |
|     1      |     1     | 0.07589718103408813  |  2   |
|     1      |     5     | 0.06052998304367065  |  3   |
|     1      |     14    | 0.05972009301185608  |  4   |
|     1      |     15    | 0.05

In [51]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23265 users and 300 items.

Data prepared in: 0.14657s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 67253.2

recommendations finished on 60000/62483 queries. users per second: 66590.7

recommendations finished on 61000/62483 queries. users per second: 66487.5

recommendations finished on 62000/62483 queries. users per second: 66447.5

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|     0      |    255    |  0.007790707624875582 |  1   |
|     0      |     2     |  0.005789128633645864 |  2   |
|     0      |    217    |  0.005641043186187744 |  3   |
|     0      |    238    |  0.005460982139293964 |  4   |
|     0      |    174    |  0.004132765990037184 |  5   |
|     0      |    248    |  0.004119442059443547 |  6   |
|     0      |     21    |  0.003964607532207782 |  7   |
|     0      |     12    | 0.0035383930573096643 |  8   |
|     0      |     35    | 0.0034239567243135893 |  9   |
|     0      |    155    | 0.0033476673639737642 |  10  |
|     1      |    251    | 0.0036339759826660156 |  1   |
|     1      |     21    | 0.0031361132860183716 |  2   |
|     1      |    105    | 0.0025331228971481323 |  3   |
|     1      |    150    | 0.0022846870124340057 |  4   |
|     1      |

In [52]:

# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23322 users and 300 items.

Data prepared in: 0.143858s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 63500.5

recommendations finished on 60000/62483 queries. users per second: 63136.5

recommendations finished on 61000/62483 queries. users per second: 62816

recommendations finished on 62000/62483 queries. users per second: 62754.4

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    132    | 3.1092607763680546 |  1   |
|     0      |     37    | 3.0159962192764436 |  2   |
|     0      |     34    | 3.0046794333457965 |  3   |
|     0      |     0     | 2.9493396427006586 |  4   |
|     0      |     27    | 2.8090853341354807 |  5   |
|     0      |    110    | 2.801204819277108  |  6   |
|     0      |     3     | 2.7957359094749443 |  7   |
|     0      |    248    | 2.7209061045979346 |  8   |
|     0      |     32    | 2.6505811050082677 |  9   |
|     0      |    230    | 2.6395693541917584 |  10  |
|     1      |    132    | 3.1090909090909093 |  1   |
|     1      |     34    | 3.023094780087473  |  2   |
|     1      |     37    | 3.0132612027814543 |  3   |
|     1      |     0     | 2.951948342797266  |  4   |
|     1      |     27    | 2.809818705959507  |  5   |
|     1   

In [53]:

# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23312 users and 300 items.

Data prepared in: 0.181932s

...

recommendations finished on 59000/62483 queries. users per second: 87378.3

recommendations finished on 60000/62483 queries. users per second: 87435

recommendations finished on 61000/62483 queries. users per second: 86862.5

recommendations finished on 62000/62483 queries. users per second: 86317.1

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     8     |  0.0  |  1   |
|     0      |    209    |  0.0  |  2   |
|     0      |    179    |  0.0  |  3   |
|     0      |    263    |  0.0  |  4   |
|     0      |     61    |  0.0  |  5   |
|     0      |    135    |  0.0  |  6   |
|     0      |     46    |  0.0  |  7   |
|     0      |     79    |  0.0  |  8   |
|     0      |    243    |  0.0  |  9   |
|     0      |    217    |  0.0  |  10  |
|     1      |    123    |  0.0  |  1   |
|     1      |     8     |  0.0  |  2   |
|     1      |    209    |  0.0  |  3   |
|     1      |    179    |  0.0  |  4   |
|     1      |    263    |  0.0  |  5   |
|     1      |     1     |  0.0  |  6   |
|     1      |     46    |  0.0  |  7   |
|     1      |     79    |  0.0  |  8   |
|     1      |    243    |  0.0  |  9   |
|     1      |    217    |  0.0  |  10  |
|     2      |    209    |  0.0  |

In [54]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23265 users and 300 items.

Data prepared in: 0.168574s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 59000/62483 queries. users per second: 65335.7

recommendations finished on 60000/62483 queries. users per second: 65013

recommendations finished on 61000/62483 queries. users per second: 64813.2

recommendations finished on 62000/62483 queries. users per second: 64639.8

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |    226    |  0.779349954406526  |  1   |
|     0      |    247    |  0.3278661666469165 |  2   |
|     0      |    230    |  0.3249692533731874 |  3   |
|     0      |    125    | 0.24846546526793603 |  4   |
|     0      |    248    |  0.2470949742343877 |  5   |
|     0      |    294    |  0.2432356828800162 |  6   |
|     0      |    204    | 0.22412753065863808 |  7   |
|     0      |    213    | 0.22256205581192287 |  8   |
|     0      |     72    |  0.2190715616167566 |  9   |
|     0      |    165    | 0.21805415468386727 |  10  |
|     1      |    226    |  0.7790841750029862 |  1   |
|     1      |    247    | 0.32770735436932436 |  2   |
|     1      |    230    |  0.3248712472897723 |  3   |
|     1      |    125    | 0.24849390528031765 |  4   |
|     1      |    248    | 0.24431818181818193 |